In [1]:
!pip install diffusers transformers accelerate safetensors torch --quiet

In [2]:
!pip install opencv-python-headless scikit-image pillow matplotlib numpy --quiet

In [3]:
import os
from PIL import Image
import torch
from diffusers import StableDiffusionUpscalePipeline
import numpy as np
import matplotlib.pyplot as plt
from skimage.metrics import structural_similarity, peak_signal_noise_ratio

In [4]:
import torch, gc
gc.collect()
torch.cuda.empty_cache()
torch.cuda.ipc_collect()

In [5]:
device = "cuda" if torch.cuda.is_available() else "cpu"

pipe_superres = StableDiffusionUpscalePipeline.from_pretrained(
    "stabilityai/stable-diffusion-x4-upscaler",
    torch_dtype=torch.float16 if device=="cuda" else torch.float32,
)
pipe_superres = pipe_superres.to(device)
pipe_superres.enable_model_cpu_offload()
pipe_superres.enable_sequential_cpu_offload()

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model_index.json:   0%|          | 0.00/485 [00:00<?, ?B/s]

Fetching 13 files:   0%|          | 0/13 [00:00<?, ?it/s]

scheduler_config.json:   0%|          | 0.00/348 [00:00<?, ?B/s]

scheduler_config.json:   0%|          | 0.00/300 [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/587 [00:00<?, ?B/s]

unet/diffusion_pytorch_model.safetensors:   0%|          | 0.00/1.89G [00:00<?, ?B/s]

vae/diffusion_pytorch_model.safetensors:   0%|          | 0.00/221M [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

`torch_dtype` is deprecated! Use `dtype` instead!


In [6]:
def upscale_diffusion(
    img: Image.Image,
    prompt=(
        "ultra detailed, photorealistic enhancement, natural skin, "
        "accurate geometry, sharp textures, clean colors, restore fine detail"
    ),
    negative_prompt=(
        "blurry, distorted face, artifacts, plastic skin, noise, "
        "over-sharpened, texture repetition"
    ),
    steps=25,
    scale=5.0
):
    pipe_superres.enable_vae_tiling()
    pipe_superres.enable_attention_slicing()

    with torch.inference_mode():
        out = pipe_superres(
            prompt=prompt,
            negative_prompt=negative_prompt,
            image=img,
            num_inference_steps=steps,
            guidance_scale=scale
        )
    return out.images[0]

In [8]:
example = resize_crop(image)
result = upscale_diffusion(example)
result

NameError: name 'example' is not defined

In [10]:
from google.colab import files
from PIL import Image
import io
import os

os.makedirs("sr_outputs", exist_ok=True)

uploaded = files.upload()

for filename, data in uploaded.items():
    img = Image.open(io.BytesIO(data)).convert("RGB")
    out = upscale_diffusion(img)
    out.save(f"sr_outputs/{filename.rsplit('.',1)[0]}_upscaled.png")



Saving 41f04a24462097.563349afeb20e (5)_out.png to 41f04a24462097.563349afeb20e (5)_out.png
Saving 62b87fa8b87daa226804525484e47b32 (1)_out.png to 62b87fa8b87daa226804525484e47b32 (1)_out.png
Saving 6183836af09a14e28d63f4bdaed4c315 (1)_out.png to 6183836af09a14e28d63f4bdaed4c315 (1)_out.png
Saving a355db95aecf9f9f05f6b3dcdbc92c53 (6)_out.png to a355db95aecf9f9f05f6b3dcdbc92c53 (6)_out.png
Saving images28 (1)_out.png to images28 (1)_out.png
Saving Johnny Hooper (1)_out.png to Johnny Hooper (1)_out.png


  0%|          | 0/25 [00:00<?, ?it/s]

  0%|          | 0/25 [00:00<?, ?it/s]

  0%|          | 0/25 [00:00<?, ?it/s]

  0%|          | 0/25 [00:00<?, ?it/s]

  0%|          | 0/25 [00:00<?, ?it/s]

  0%|          | 0/25 [00:00<?, ?it/s]